In [ ]:
# 3. Google Drive Bağlantısı
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. Gerekli Kütüphaneleri İçe Aktarma
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from functools import partial
from timm.models.vision_transformer import VisionTransformer, _cfg
from timm.models.layers import trunc_normal_

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
# Veri seti yolu
data_dir = "/content/drive/MyDrive/Dataset"  # Kendi veri seti dizininizi girin

In [ ]:
# 4. Veri Seti Hazırlığı
batch_size = 32

# Transformations
common_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Veri setlerini yükleme
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=common_transform)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/validation", transform=common_transform)
test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=common_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# PiT modelini tanımla
class PiT(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768, depth=12, num_heads=12, num_classes=2, mlp_ratio=4.0, dropout=0.1):
        super().__init__()
        self.patch_embed = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, (img_size // patch_size) ** 2 + 1, embed_dim))
        self.dropout = nn.Dropout(dropout)
        self.blocks = nn.ModuleList([self._make_block(embed_dim, num_heads, mlp_ratio, dropout) for _ in range(depth)])
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def _make_block(self, embed_dim, num_heads, mlp_ratio, dropout):
        return nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout),
            nn.LayerNorm(embed_dim),
            nn.Sequential(
                nn.Linear(embed_dim, int(embed_dim * mlp_ratio)),
                nn.GELU(),
                nn.Linear(int(embed_dim * mlp_ratio), embed_dim),
                nn.Dropout(dropout)
            )
        )

    def forward(self, x):
        B, _, _, _ = x.shape
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2)
        cls_token = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        x = x + self.pos_embed
        x = self.dropout(x)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)
        return self.head(x[:, 0])



In [ ]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim

# 1. PiT Modelini İndirme ve Sınıflandırıcı Katmanını Ayarlama
num_classes = len(train_dataset.classes)  # Sınıf sayısını veri setinden alıyoruz
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Timm ile PiT modelini yükleme
model = timm.create_model('pit_ti_224', pretrained=True)  # PiT Tiny modelini kullanıyoruz
model.head = nn.Linear(model.head.in_features, num_classes)  # Sınıflarınıza göre son katmanı değiştiriyoruz
model = model.to(device)

# 2. Loss (Kayıp Fonksiyonu)
criterion = nn.CrossEntropyLoss()

# 3. Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# 4. Learning Rate Scheduler (isteğe bağlı)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Öğrenme hızını azaltmak için


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/19.4M [00:00<?, ?B/s]

In [ ]:
# Eğitim döngüsü
from tqdm.auto import tqdm
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss, correct = 0, 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_dataset):.4f}")


  0%|          | 0/485 [00:00<?, ?it/s]

Epoch [1/10], Loss: 1.1488, Accuracy: 0.4700


  0%|          | 0/485 [00:00<?, ?it/s]

Epoch [2/10], Loss: 1.0023, Accuracy: 0.5371


  0%|          | 0/485 [00:00<?, ?it/s]

...

Epoch [8/10], Loss: 0.4073, Accuracy: 0.8404


  0%|          | 0/485 [00:00<?, ?it/s]

Epoch [9/10], Loss: 0.3336, Accuracy: 0.8713


  0%|          | 0/485 [00:00<?, ?it/s]

Epoch [10/10], Loss: 0.2880, Accuracy: 0.8921


In [ ]:
# Modeli kaydet
model_path = '/content/drive/MyDrive/pit_model.pth'
torch.save(model.state_dict(), model_path)
print(f"Model {model_path} yolunda kaydedildi.")


Model /content/drive/MyDrive/pit_model.pth yolunda kaydedildi.


In [ ]:
# Test döngüsü
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        test_loss += criterion(outputs, labels).item()
        correct += (outputs.argmax(1) == labels).sum().item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {correct/len(test_dataset):.4f}")

Test Loss: 1.4177, Test Accuracy: 0.5941
